Interfacing with some native libraries (for example written in C or C++) can necessitate writing native callbacks to provide business logic to the library. The `numba.cfunc()` decorator creates a compiled function callable from foreign C code, using the signature of your choice.

## 1. Basic usage

The `@cfunc` decorator has a similar usage to `@jit`, but with an important difference: passing a single signature is mandatory. It determines the visible signature of the C callback:

In [ ]:
from numba import cfunc

@cfunc("float64(float64, float64)")
def add(x, y):
    return x + y

The C function object exposes the address of the compiled C callback as the `address` attribute, so that you can pass it to any foreign C or C++ library. It also exposes a `ctypes` callback object pointing to that callback; that object is also callable from Python, making it easy to check the compiled code:

## 2. Example

In this example, we are going to be using the `scipy.integrate.quad` function. That function accepts either a regular Python callback or a C callback wrapped in a ctypes callback object.

Let’s define a pure Python integrand and compile it as a C callback:

In [1]:
import numpy as np
import scipy.integrate as si
from numba import cfunc

def intergrand(t):
    return np.exp(-t) / (t ** 2)

nb_intergrand = cfunc("float64(float64)")(intergrand)

def do_intergrate(func):
    return si.quad(func, 1, np.inf)

print(do_intergrate(intergrand))
print(do_intergrate(nb_intergrand.ctypes))

(0.14849550677592208, 3.8736750295955804e-10)
(0.14849550677592208, 3.8736750295955804e-10)


Using the compiled callback, the integration function does not invoke the Python interpreter each time it evaluates the integrand

In [2]:
%timeit -r 10 -n 5 do_intergrate(intergrand)

188 µs ± 61.3 µs per loop (mean ± std. dev. of 10 runs, 5 loops each)


In [3]:
%timeit -r 10 -n 5 do_intergrate(nb_intergrand.ctypes)

12.4 µs ± 3.87 µs per loop (mean ± std. dev. of 10 runs, 5 loops each)


## 3. Dealing with pointers and array memory

A less trivial use case of C callbacks involves doing operation on some array of data passed by the caller. As C doesn’t have a high-level abstraction similar to Numpy arrays, the C callback’s signature will pass low-level pointer and size arguments. Nevertheless, the Python code for the callback will expect to exploit the power and expressiveness of Numpy arrays.

In the following example, the C callback is expected to operate on 2-d arrays, with the signature `void(double *input, double *output, int m, int n)`. You can implement such a callback thusly:

In [4]:
import numpy as np
import ctypes
from numba import cfunc, types, carray
from scipy import ndimage, LowLevelCallable

c_sig = types.intc(types.CPointer(types.intp), 
                   types.CPointer(types.double), 
                   types.intc, 
                   types.intc, 
                   types.voidptr)

@cfunc(c_sig)
def transform(output_ptr, input_ptr, output_rank, input_rank, user_data):
    input_ = carray(input_ptr, shape=(input_rank, ))
    output_ = carray(output_ptr, shape=(output_rank, ))
    shift = carray(user_data, shape=(1, ), dtype=types.double)[0]
    for i in range(input_rank):
        input_[i] = output_[i] - shift
    return 1

shift = 0.5

user_data = ctypes.c_double(shift)
ptr = ctypes.cast(ctypes.pointer(user_data), ctypes.c_void_p)
callback = LowLevelCallable(transform.ctypes, user_data=ptr)

im = np.arange(12).reshape(4, 3).astype(np.float64)

In [5]:
ndimage.geometric_transform(im, callback)

ValueError: Invalid scipy.LowLevelCallable signature "long (longlong *, double *, long, long, void *)". Expected one of: ['int (intptr_t *, double *, int, int, void *)', 'int (npy_intp *, double *, int, int, void *)', 'int (long long *, double *, int, int, void *)']

The `numba.carray()` function takes as input a data pointer and a shape and returns an array view of the given shape over that data. The data is assumed to be laid out in C order. If the data is laid out in Fortran order, `numba.farray()` should be used instead.

In [ ]:
import numpy as np
from numba import cfunc, carray
from numba.types import intc, intp, float64, voidptr,CPointer
from scipy import ndimage, LowLevelCallable

c_sig = intc(CPointer(float64), intp, CPointer(float64), voidptr)

@cfunc(c_sig)
def nbmin(values_ptr, len_values, result, data):
    values = carray(values_ptr, shape=(len_values, ), dtype=float64)
    result[0] = np.inf
    for v in values:
        if v < result[0]:
            result[0] = v
    return 1        

image = np.random.random((2048, 2048))
footprint = np.array([[0, 1, 0], 
                      [1, 1, 0],
                      [0, 1, 0]], dtype=np.bool)
callback = LowLevelCallable(nbmin.ctypes)

In [ ]:
ndimage.generic_filter(image, function=callback, footprint=footprint)

## 4. Handling C structures

### 4.1 With CFFI

For applications that have a lot of state, it is useful to pass data in C structures. To simplify the interoperability with C code, numba can convert a `cffi` type into a numba `Record` type using `numba.cffi_support.map_type`:

In [ ]:
from numba import cffi_support

nbtype = cffi_support.map_type(cffi_type)